In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 

from datetime import datetime

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
sys.path

['/opt/miniconda3/envs/RecSys/lib/python312.zip',
 '/opt/miniconda3/envs/RecSys/lib/python3.12',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/lib-dynload',
 '',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/setuptools/_vendor',
 '/Users/safuan/Python/RecSysCompetition2024Polimi/libs',
 '/Users/safuan/Python/RecSysCompetition2024Polimi',
 '/Users/safuan/Python/RecSysCompetition2024Polimi/src']

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [5]:
from libs.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from operator import itemgetter
import numpy as np

from src.utils import tf_idf, train_model, write_submission

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")


In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions


In [7]:
results_cf = pd.DataFrame()
save_path_cf = "result_experiments/ItemKNNCFRecommender/"

results_cbf = pd.DataFrame()
save_path_cbf = "result_experiments/ItemKNNCBFRecommender/"

In [14]:
if os.path.exists(save_path_cf+"results.csv"):
    results = pd.read_csv(save_path_cf+"results.csv")
else:
    os.makedirs(save_path_cf)

if os.path.exists(save_path_cbf+"results.csv"):
    results = pd.read_csv(save_path_cbf+"results.csv")
else:
    os.makedirs(save_path_cbf)

In [10]:
max_MAP_cf = -1
max_MAP_cbf = -1

In [13]:
for similarity in ['cosine','pearson','jaccard','tanimoto','adjusted']:
    for topK in [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 50, 100]:
        for shrink in [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 50, 100]:
            for normalize in [True, False]:
                current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                params = {
                    "similarity": similarity, 
                    "topK": topK,
                    "shrink": shrink,
                    "normalize": normalize,
                    "time": current_timestamp
                }
                
                recommender_cf = ItemKNNCFRecommender(URM_train)
                recommender_cf.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=normalize)
    
                metrics_df, _ = evaluator_validation.evaluateRecommender(recommender_cf)
                if max_MAP_cf < metrics_df["MAP"][10]:
                    max_MAP_cf = metrics_df["MAP"][10]
                print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP_cf)
                      
                params_df = pd.DataFrame([params])
                iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
                results_cf = pd.concat([results_cf, iteration_results], ignore_index=True)
                results_cf.to_csv(save_path_cf+"results.csv", index=False)

Similarity column 38121 (100.0%), 4907.73 column/sec. Elapsed time 7.77 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 8.76 sec. Users per second: 4061
similarity: cosine, topK: 5, shrink: 0, normalize: True, time: 2024-12-04 23:42:02 MAP:  0.05091123149847265 max MAP:  0.05123910077153919
Similarity column 38121 (100.0%), 5744.48 column/sec. Elapsed time 6.64 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 9.90 sec. Users per second: 3592
similarity: cosine, topK: 5, shrink: 0, normalize: False, time: 2024-12-04 23:42:18 MAP:  0.03363353983915828 max MAP:  0.05123910077153919
Similarity column 38121 (100.0%), 4693.78 column/sec. Elapsed time 8.12 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 9.20 sec. Users per second: 3867
similarity: cosine, topK: 5, shrink: 5, normalize: True, time: 2024-12-04 23:42:35 MAP:  0.05123910077153919 max MAP:  0.05123910077153919
Similarity column 38121 (100.0%), 5226.62 column/sec. Elapsed time 7.29 sec
EvaluatorHoldout: Processed 35573 (100.0%

In [ ]:
for similarity in ['cosine','pearson','jaccard','tanimoto','adjusted']:
    for topK in [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 50, 100]:
        for shrink in [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 50, 60, 70, 80, 100]:
            for normalize in [True, False]:
                current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                params = {
                    "similarity": similarity, 
                    "topK": topK,
                    "shrink": shrink,
                    "normalize": normalize,
                    "time": current_timestamp
                }
                
                recommender_cbf = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
                recommender_cbf.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=normalize)
    
                metrics_df, _ = evaluator_validation.evaluateRecommender(recommender_cbf)
                if max_MAP_cbf < metrics_df["MAP"][10]:
                    max_MAP_cbf = metrics_df["MAP"][10]
                print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP_cbf)
                      
                params_df = pd.DataFrame([params])
                iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
                results_cbf = pd.concat([results_cbf, iteration_results], ignore_index=True)
                results_cbf.to_csv(save_path_cbf+"results.csv", index=False)

Similarity column 38121 (100.0%), 1168.35 column/sec. Elapsed time 32.63 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 8.96 sec. Users per second: 3968
similarity: cosine, topK: 5, shrink: 0, normalize: True, time: 2024-12-05 17:13:05 MAP:  0.012685135730282687 max MAP:  0.015102142966464551
Similarity column 38121 (100.0%), 1283.99 column/sec. Elapsed time 29.69 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 8.92 sec. Users per second: 3988
similarity: cosine, topK: 5, shrink: 0, normalize: False, time: 2024-12-05 17:13:47 MAP:  0.01181182535889789 max MAP:  0.015102142966464551
Similarity column 38121 (100.0%), 1205.35 column/sec. Elapsed time 31.63 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 9.34 sec. Users per second: 3807
similarity: cosine, topK: 5, shrink: 5, normalize: True, time: 2024-12-05 17:14:25 MAP:  0.015102142966464551 max MAP:  0.015102142966464551
Similarity column 38121 (100.0%), 1196.18 column/sec. Elapsed time 31.87 sec
EvaluatorHoldout: Processed 3557

In [17]:
max_tuple = max(MAP_CF, key=itemgetter(3))
print('The best combination for the ICF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

max_tuple = max(MAP_CBF, key=itemgetter(3))
print('The best combination for the CBF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))


The best combination for the ICF recommender is (similarity=jaccard,topK=10,shrink=10), with a MAP = 0.050442041349481284
The best combination for the CBF recommender is (similarity=jaccard,topK=10,shrink=50), with a MAP = 0.0149851419267978


In [9]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(topK=10, shrink=10, similarity='jaccard', normalize=False)

result_dict, _ = evaluator_validation.evaluateRecommender(recommender)


Similarity column 38121 (100.0%), 5006.58 column/sec. Elapsed time 7.61 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 8.90 sec. Users per second: 3998


In [2]:
import pandas as pd

In [19]:
result_dict["MAP"][10]

0.05107207901341492

In [26]:
recommender.recommend(1000, cutoff=10)

([6842, 6374, 2, 6853, 2836, 21442, 6852, 6858, 19208, 6611],
 array([[0.        , 0.        , 0.44027978, ..., 0.        , 0.        ,
         0.        ]], dtype=float32))

In [32]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [35]:
write_submission_lib_model(recommender, filename='cbf_urm_lib.csv')